In [91]:
from llama_index import GPTSimpleVectorIndex, GPTChromaIndex, SimpleDirectoryReader
from dotenv import load_dotenv

load_dotenv()

True

In [75]:
!echo PyPDF2 >> requirements.txt

# GPT text to query
l'origine est un texte qui va se transfomer 

In [76]:
# documents = SimpleDirectoryReader("news").load_data()
# index = GPTSimpleVectorIndex.from_documents(documents)


In [15]:
# index.save_to_disk('vectors/simple_index.json')

# GPT vector extraction to query

## gpt fee

In [2]:
from llama_index import GPTSimpleVectorIndex, GPTChromaIndex, SimpleDirectoryReader
from dotenv import load_dotenv

load_dotenv()
index = GPTSimpleVectorIndex.load_from_disk('vectors/simple_index.json')

## local fee (free)

In [5]:
r = index.query(' what is parseBytes32String ?')
print(r)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2323 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 7 tokens



parseBytes32String is not mentioned in the context information.


# Use chromadb for embedding and querying

In [92]:
from llama_index import GPTChromaIndex, SimpleDirectoryReader
from dotenv import load_dotenv
import chromadb

load_dotenv()

client = chromadb.Client(

)
chroma_collection = client.create_collection('solidity')
print(chroma_collection.count()) # expect 0

documents = SimpleDirectoryReader("news").load_data()
print(chroma_collection.count()) # expect 0

INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:chromadb:Running Chroma using direct local API.
INFO:chromadb.db.duckdb:PersistentDuckDB del, about to run persist
INFO:chromadb.db.duckdb:Persisting DB to disk, putting it in the save folder: ./chroma
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


0
0


In [93]:
from chromadb.config import Settings
from chromadb.utils import embedding_functions

client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./chroma" # Optional, defaults to .chromadb/ in the current directory
))

chroma_collection = client.create_collection('solidity')





INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:chromadb:Running Chroma using direct local API.
INFO:chromadb.db.duckdb:loaded in 0 embeddings
INFO:chromadb.db.duckdb:loaded in 0 collections
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu
INFO:chromadb.db.duckdb:Exiting: Cleaning up .chroma directory


In [94]:
# collection = client.get_collection(name="solidity")


## gpt fee

In [95]:
index = GPTChromaIndex.from_documents(documents, chroma_collection=chroma_collection)


INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1102164 tokens
INFO:chromadb.db.duckdb:Exiting: Cleaning up .chroma directory


302


AttributeError: 'Collection' object has no attribute 'persist'

In [96]:
print(chroma_collection.count())
client.persist()
index.save_to_disk("./vectors/solidity--cheetsheet_solidity-documentation--gpt3-chromadb2.json")

INFO:chromadb.db.duckdb:Persisting DB to disk, putting it in the save folder: ./chroma


302


## local fee

In [97]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = client.get_collection(name="solidity", embedding_function=sentence_transformer_ef, )

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [98]:
index = GPTChromaIndex.load_from_disk('vectors/solidity--cheetsheet_solidity-documentation--gpt3-chromadb.json', chroma_collection=collection)
# index = GPTChromaIndex.load_from_disk('')

In [107]:

question = """ what is a public function ? does it have a default method ? which one ?
"""

# start_time = time.time()
r = index.query(question)
print(r)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 4146 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 17 tokens




A public function is a function that can be called by any account and is defined by the developer. It does not have a default method; instead, developers can use a variety of tools such as IntelliJ IDEA plugin, Sublime Text, Vim, Visual Studio Code (VS Code), and Ethereum Remix Visual Studio Code extension to define the function's behavior. Additionally, Solidity Visual Studio Code extension, Solidity Visual Auditor extension, and Truffle for VS Code can be used to add security centric syntax and semantic highlighting to Visual Studio Code.




A payable function is a function that accepts Ether. The state mutability of a payable function is specified as payable. For example, in the document 67b2b13c-8029-47b4-a760-bb526eaae64b, page 124, it states that "stateMutability : a string with one of the following values: pure(specified to not read blockchain state ), view(specified to not modify the blockchain state ),nonpayable (function does not accept Ether - the default) andpayable (function accepts Ether). To avoid reentrancy, you can use the Checks-Effects-Interactions pattern as demonstrated in the example code provided. This pattern ensures that all code paths through a contract complete all required checks of the supplied parameters before modifying the contract’s state, and only then it makes any changes to the state. It may make calls to functions in other contracts afterall planned state changes have been written to storage. This is a common foolproof way to prevent reentrancy attacks.


In [ ]:
# start_time = time.time()
r = index.query(question)
# end_time = time.time()

# execution_time = end_time - start_time

# print("Execution time:", execution_time, "seconds")
# print("response:", response)

In [34]:
chroma_collection.peek()

{'ids': ['de19054e-18f1-4212-a0f5-cf861f870b9f',
  'f6eeb79f-35c2-459c-8fb8-9a0e4692e158',
  'd7eca50f-5910-4338-af47-12054ca5bcb6',
  '6a400160-61a2-4b2b-a70f-ae3cee9db724',
  '425058c0-700e-48dc-8fee-7fbe18075011',
  'eb59e7c9-42c8-4b55-9f3f-04286d45d0b5',
  '1a88d30e-a336-4a46-8898-26c880b28ac1',
  '5bda1a61-4b79-404a-8b41-dea5a7aecb95',
  '4f5f435a-92f1-4366-946e-157e28c4325e',
  '662fa66c-4f69-4f0e-9c93-0ce5a85b190b'],
 'embeddings': [[-0.0017868319991976023,
   0.00888834334909916,
   -0.01996176689863205,
   -0.0020018157083541155,
   -0.0018414589576423168,
   0.010798526927828789,
   -0.010185294784605503,
   -0.005022160708904266,
   0.004662679973989725,
   -0.013237359002232552,
   0.02273893542587757,
   0.03448197990655899,
   -0.01361093670129776,
   -0.012271693907678127,
   -0.016648903489112854,
   0.009882202371954918,
   0.008268062956631184,
   0.005483847111463547,
   0.025685269385576248,
   -0.033720728009939194,
   -0.0007753511308692396,
   0.02103316225111484

# Using GPT Transformer ? (completely free but GPU needed)

In [12]:
from sentence_transformers import SentenceTransformer

In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cpu


In [32]:
sentences = [
    'This frameworks generates embeddings for each sentence (not words!)',
    'Each sentence will be encoded, or mapped to a 384 dimensional vector space',
    'Supertype.ai helps companies productionized their machine learning systemps',
    'Algorit.ma helps companies train and upskill their datascience teams'
]

embeddings = model.encode(sentences)
for sentence, embedding in zip(sentences, embeddings):
    print("sentence:", sentence)
    print("embedding:", embedding)
    print("\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

sentence: This frameworks generates embeddings for each sentence (not words!)
embedding: [ 5.89194056e-03 -5.71730956e-02  3.21087148e-03  2.15804260e-02
  7.04285055e-02  8.83549824e-02  4.73793969e-02 -5.06044254e-02
  1.43914344e-02 -3.35150883e-02  6.99151158e-02 -2.54673995e-02
  7.58863017e-02  3.31189819e-02 -9.23741534e-02  7.84222633e-02
  9.55355447e-03  7.14126751e-02 -7.35655725e-02 -6.10696003e-02
  7.11640285e-04  7.15635493e-02  6.40391260e-02 -6.18250147e-02
  9.76015907e-03  4.36330251e-02 -8.85096490e-02  2.34044809e-02
  9.76139307e-02  1.97690614e-02  3.80922556e-02 -9.62194335e-03
 -1.23841623e-02  8.47686157e-02  3.70269679e-02  2.11642701e-02
 -1.75945845e-03  3.44776139e-02 -2.86426637e-02  3.53673939e-03
 -4.66161445e-02  5.55574056e-03 -6.88997388e-04  6.34804964e-02
  9.80474278e-02 -3.25958841e-02 -9.19352844e-02 -8.32318678e-04
 -4.11938094e-02 -3.23233730e-03 -1.02416672e-01 -4.97396030e-02
 -2.70244461e-02 -1.41957309e-03 -2.49596927e-02  1.68586373e-02
 

In [67]:
index = GPTChromaIndex.load_from_disk('vectors/solidity--cheetsheet_solidity-documentation--gpt3-chromadb.json', chroma_collection=)
# index = GPTChromaIndex.load_from_disk('')

ValueError: Missing chroma collection!